### Does Age Have a Significant Effect on Performance

In [1]:
import sqlite3
import pandas as pd
import math
from scipy import stats
import seaborn as sns
import numpy as np

In [2]:
connection = sqlite3.connect('Northwind_small.sqlite')
cursor = connection.cursor()
#orderDetail_df = sql_select_to_df("""select * from OrderDetail;""")

In [3]:
def sql_select_to_df(SQL_COMMAND, cursor=cursor):
    results = cursor.execute(SQL_COMMAND).fetchall()
    df = pd.DataFrame(results)
    df.columns = [i[0] for i in cursor.description]
    return df

In [4]:
orders_employees_df = sql_select_to_df("""SELECT O.EmployeeId, E.BirthDate, COUNT(O.Id) AS Total_Orders  
                                FROM [Order] AS O
                                JOIN Employee as E
                                ON O.EmployeeId = E.Id
                                GROUP BY O.EmployeeId
                        """)

In [5]:
#What year is the dataset dated?

order_dates = sql_select_to_df("""SELECT ShippedDate FROM [Order]""")
print(order_dates)

dataset_yr = 2014

hiredates = sql_select_to_df("""SELECT HireDate FROM Employee""")

    ShippedDate
0    2012-07-16
1    2012-07-10
2    2012-07-12
3    2012-07-15
4    2012-07-11
5    2012-07-16
6    2012-07-23
7    2012-07-15
8    2012-07-17
9    2012-07-22
10   2012-07-23
11   2012-07-25
12   2012-07-29
13   2012-07-30
14   2012-07-25
15   2012-07-31
16   2012-08-23
17   2012-08-12
18   2012-07-31
19   2012-08-06
20   2012-08-02
21   2012-08-09
22   2012-08-02
23   2012-08-30
24   2012-08-06
25   2012-08-12
26   2012-08-16
27   2012-08-09
28   2012-08-14
29   2012-08-13
..          ...
800  2014-04-30
801  2014-05-04
802  2014-05-05
803        None
804  2014-05-01
805  2014-04-29
806        None
807  2014-05-05
808  2014-05-01
809  2014-05-01
810        None
811        None
812  2014-05-04
813        None
814        None
815  2014-05-06
816  2014-05-04
817        None
818  2014-05-04
819  2014-05-06
820        None
821  2014-05-06
822        None
823        None
824        None
825        None
826        None
827        None
828        None
829        None

[830 ro

In [6]:
orders_employees_df
#order_employees_df['Age']

birth_dates = orders_employees_df['BirthDate'].values

dates = []
for b in birth_dates:
    split_date = b.split('-')
    dates.append(split_date)

years = []
for d in dates:
    yr = int(d[0])
    years.append(yr)

ages = []
for y in years:
    age = dataset_yr - y
    ages.append(age)
    
sorted(ages)

[16, 19, 19, 22, 24, 27, 30, 34, 45]

In [7]:
orders_employees_df['Age'] = ages
orders_employees_df#

orders_age_df = orders_employees_df.iloc[:,2:]

under_twenties = orders_age_df.loc[orders_age_df['Age'] <20]
under_twenties_orders = under_twenties['Total_Orders'].values

twenties = orders_age_df.loc[(orders_age_df['Age'] >= 20) & (orders_age_df['Age'] < 30)]
twenties_orders = twenties['Total_Orders'].values

thirties_plus = orders_age_df.loc[(orders_age_df['Age'] >= 30)]
thirties_plus_orders = thirties_plus['Total_Orders'].values

print('population 1 (order figures of employees under age 20): ' + str(under_twenties_orders))
print('population 2 (order figures of employees between ages 20-29): ' + str(twenties_orders))
print('population 3 (order figures of employees of age 30 and over): ' + str(thirties_plus_orders))

num_of_groups = 3

population 1 (order figures of employees under age 20): [127  67  43]
population 2 (order figures of employees between ages 20-29): [ 42  72 104]
population 3 (order figures of employees of age 30 and over): [123  96 156]


In [8]:
grand_mean_numer = np.sum(under_twenties_orders) + np.sum(twenties_orders) + np.sum(thirties_plus_orders)
grand_mean_denom = len(under_twenties_orders) + len(twenties_orders) + len(thirties_plus_orders)
#len(under_twenties_orders)
grand_mean = grand_mean_numer/grand_mean_denom
grand_mean

mean_1 = np.mean(under_twenties_orders)
mean_2 = np.mean(twenties_orders)
mean_3 = np.mean(thirties_plus_orders)

print(mean_1, mean_2, mean_3, grand_mean)
#sst = 

79.0 72.66666666666667 125.0 92.22222222222223


In [9]:
def get_sst(pop1, pop2, pop3):
    sst = []
    
    for i in pop1:
        sst_component_1 = (i - grand_mean)**2
        sst.append(sst_component_1)
        
    for j in pop2:
        sst_component_2 = (j - grand_mean)**2
        sst.append(sst_component_2)
        
    for k in pop3:
        sst_component_3 = (k - grand_mean)**2
        sst.append(sst_component_3)
    
    return sum(sst)

get_sst(under_twenties_orders, twenties_orders, thirties_plus_orders)

12367.555555555555

In [10]:
def get_ssw(pop1, pop2, pop3):
    ssw = []
    
    for i in pop1:
        ssw_component_1 = (i - mean_1)**2
        ssw.append(ssw_component_1)
        
    for j in pop2:
        ssw_component_2 = (j - mean_2)**2
        ssw.append(ssw_component_2)
        
    for k in pop3:
        ssw_component_3 = (k - mean_3)**2
        ssw.append(ssw_component_3)
    
    return sum(ssw)

ssw = get_ssw(under_twenties_orders, twenties_orders, thirties_plus_orders)

def get_ssw_degrees_of_freedom(m,n):    
    return m*(n-1)

ssw_degrees_freedom = get_ssw_degrees_of_freedom(num_of_groups, len(under_twenties_orders))
ssw_degrees_freedom

6

In [11]:
def get_ssb(pop1, pop2, pop3):
    ssb = []
    
    for i in pop1:
        ssb_component_1 = (mean_1 - grand_mean)**2
        ssb.append(ssb_component_1)
        
    for j in pop2:
        ssb_component_2 = (mean_2 - grand_mean)**2
        ssb.append(ssb_component_2)
        
    for k in pop3:
        ssb_component_3 = (mean_3 - grand_mean)**2
        ssb.append(ssb_component_3)
    
    return sum(ssb)

def get_ssb_degrees_of_freedom(m):    
    return m-1

ssb = get_ssb(under_twenties_orders, twenties_orders, thirties_plus_orders)
ssb_degrees_freedom = get_ssb_degrees_of_freedom(num_of_groups)

Null hypothesis: Age doesn't make a difference to quantity sold(mu1 == mu2 == mu3) <br>
Alternate hypothesis: Age does make a difference to quantity sold (mu1 == mu2 == mu3)

In [12]:
def get_f_statistic(ss_between, df_between, ss_within, df_within):
        numerator = ss_between/df_between
        denominator = ss_within/df_within
        return numerator/denominator

    
alpha = 0.05
f_statistic = get_f_statistic(ssb, ssb_degrees_freedom, ssw, ssw_degrees_freedom)

critical_f_value =  5.14

print(f_statistic)

1.9651173164421445


fail to reject Null Hypothesis